In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33468
2,Huelva,Confirmados PDIA 14 días,883
3,Huelva,Tasa PDIA 14 días,"172,06773583802638"
4,Huelva,Confirmados PDIA 7 días,378
5,Huelva,Total Confirmados,33653
6,Huelva,Curados,31836
7,Huelva,Fallecidos,390


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33468.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9166.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1320 personas en los últimos 7 días 

Un positivo PCR cada 486 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33468.0,378.0,883.0,513170.0,73.659801,172.067736,33.0
Huelva-Costa,19293.0,236.0,508.0,289548.0,81.506348,175.445867,27.0
Huelva (capital),9166.0,109.0,296.0,143837.0,75.780223,205.788497,18.0
Ayamonte,1362.0,28.0,30.0,21104.0,132.676270,142.153146,4.0
Sierra de Huelva-Andévalo Central,2946.0,23.0,64.0,67391.0,34.129186,94.968171,4.0
Aljaraque,1181.0,19.0,27.0,21474.0,88.479091,125.733445,2.0
Condado-Campiña,10875.0,118.0,308.0,156231.0,75.529184,197.143973,2.0
Aroche,94.0,3.0,3.0,3054.0,98.231827,98.231827,1.0
Cartaya,1412.0,27.0,48.0,20083.0,134.442065,239.008116,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Juan del Puerto,593.0,17.0,39.0,9411.0,180.639677,414.408671,0.0
Cerro de Andévalo (El),156.0,2.0,9.0,2327.0,85.947572,386.764074,0.0
Puebla de Guzmán,142.0,1.0,9.0,3092.0,32.341527,291.073739,NaN
Manzanilla,109.0,4.0,6.0,2118.0,188.857413,283.286119,0.0
Palos de la Frontera,731.0,17.0,32.0,11742.0,144.779424,272.525975,0.0
Moguer,1414.0,28.0,59.0,21867.0,128.046829,269.812960,-1.0
Paymogo,78.0,3.0,3.0,1162.0,258.175559,258.175559,0.0
Valverde del Camino,666.0,7.0,32.0,12750.0,54.901961,250.980392,1.0
Bonares,304.0,10.0,15.0,6060.0,165.016502,247.524752,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Trigueros,334.0,1.0,9.0,7862.0,12.719410,114.474688,0.0,0.111111
Puebla de Guzmán,142.0,1.0,9.0,3092.0,32.341527,291.073739,0.0,0.111111
Lucena del Puerto,268.0,1.0,7.0,3261.0,30.665440,214.658080,0.0,0.142857
Valverde del Camino,666.0,7.0,32.0,12750.0,54.901961,250.980392,1.0,0.218750
Cerro de Andévalo (El),156.0,2.0,9.0,2327.0,85.947572,386.764074,0.0,0.222222
San Bartolomé de la Torre,308.0,1.0,4.0,3761.0,26.588673,106.354693,0.0,0.250000
Almonte,1919.0,13.0,47.0,24507.0,53.046068,191.781940,0.0,0.276596
Gibraleón,655.0,4.0,14.0,12737.0,31.404569,109.915993,0.0,0.285714
Bollullos Par del Condado,1038.0,6.0,19.0,14387.0,41.704316,132.063669,0.0,0.315789
